In [1]:
from dotenv import load_dotenv
from groq import Groq
import json
import os
import random

load_dotenv()

True

In [2]:
# a function to convert dates to years
def days_to_years_func(days: int = 0) -> float:
    """converts days into years"""
    return days / 365


def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:  # noqa
        return json.dumps({"error": "Invalid expression"})

In [3]:
client = Groq(
    api_key=os.environ["GROQ_API_KEY"],
)

client2 = Groq(
    api_key="hh",
    base_url="http://localhost:8000/",
)
MODEL = "llama3-groq-70b-8192-tool-use-preview"
# MODEL='llama3-8b-8192'
# MODEL='llama-3.1-70b-versatile'


tools = [
    {
        "type": "function",
        "function": {
            "name": "days_to_years_func",
            "description": "converts days to years",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "int",
                        "description": "number of days, e.g. 245",
                    }
                },
                "required": ["days"],
            },
        },
    }
]

In [4]:
def call_stream(stream=False, client=client, prompt=None, printt=True):
    print("stream: ", stream)

    if not prompt:
        random_words = [
            "potato",
            "tomato",
            "banana",
            "rail",
            "railed",
            "nutt",
            "bolt",
            "go",
            "Elephant",
        ]
        word = random.sample(random_words, 1)[0]
        prompt = "spell " + word

    print(f"prompt: {prompt}")
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=MODEL,
        stream=stream,
    )

    if printt:
        if stream:
            for chunk in chat_completion:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
        else:
            print(chat_completion.choices[0].message.content)
    else:
        return chat_completion

In [5]:
call_stream(stream=True, client=client, printt=True)

stream:  True
prompt: spell bolt


 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0c2010>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0c2010>
B-O-L-T

In [6]:
call_stream(False, client, printt=True)

stream:  False
prompt: spell tomato
 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0d4e90>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0d4e90>
t-o-m-a-t-o


In [7]:
out = call_stream(False, client2, printt=True)

stream:  False
prompt: spell potato


 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0dff50>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0dff50>
p-o-t-a-t-o


In [8]:
out2 = call_stream(True, client2, printt=True)

stream:  True
prompt: spell banana
 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c099750>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c099750>
B-A-N-A-N-A

In [9]:
def run_conversation(user_prompt, client=client):
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results.",
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "days_to_years_func",
                "description": "converts days to years",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "day": {
                            "type": "int",
                            "description": "number of days, e.g. 245",
                        }
                    },
                    "required": ["days"],
                },
            },
        },
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=500,
        temperature=0.88,
        top_p=0.77,
        stream=False,
    )
    print(response)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
            "days_to_years_func": days_to_years_func,
        }

        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = str(function_to_call(**function_args))
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(model=MODEL, messages=messages)
        print(second_response)
        return second_response.choices[0].message.content

In [10]:
# user_prompt = "calculate 3*25-55*2/25-4-2?"
user_prompt = "i have been running for  4312 days , what would that be in years"
response = run_conversation(user_prompt)

 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0d6210>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x72524c0d6210>
ChatCompletion(id='chatcmpl-7ffabe54-9cd3-473f-8997-0a22d365bfa7', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_t9ya', function=Function(arguments='{"days": 4312}', name='days_to_years_func'), type='function')]))], created=1722155525, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=CompletionUsage(completion_tokens=29, prompt_tokens=309, total_tokens=338, completion_time=0.089045641, prompt_time=0.022945398, queue_time=None, total_time=0.11199103899999999), x_groq={'id': 'req_01j3w7fsjhe76b897hmjvy90t6'})
 respnse 

In [30]:
# user_prompt = "calculate 3*25-55*2/25-4-2?"
user_prompt = "i have been running for  4832 days , what would that be in years"
response = run_conversation(user_prompt, client2)

 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x725246b44150>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x725246b44150>
ChatCompletion(id='chatcmpl-3bbbd52d-1819-40e1-9820-d74c9add04a8', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vnr1', function=Function(arguments='{"days": 4832}', name='days_to_years_func'), type='function')]))], created=1722155599, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=CompletionUsage(completion_tokens=29, prompt_tokens=309, total_tokens=338, completion_time=0.088948484, prompt_time=0.024025886, queue_time=None, total_time=0.11297436999999999), x_groq={'id': 'req_01j3w7j17qen9s6md09cg4jawd'})
 respnse 

In [12]:
days_to_years_func(4352)

11.923287671232877

In [13]:
(3 * 25) - (55 * 2 / 25) - 6

64.6

In [14]:
from pydantic import BaseModel
from typing import Dict, Any

In [15]:
class B(BaseModel):
    d: dict


class A(BaseModel):
    b: list[B]
    c: Dict[str, B]
    e: str | None = None

In [16]:
a = A(b=[B(d={"hi": "by"})], c={"another": B(d={"no": "yes"})})

In [17]:
aa = a.model_dump()
aaa = a.model_dump_json()

In [18]:
aa, aaa

({'b': [{'d': {'hi': 'by'}}],
  'c': {'another': {'d': {'no': 'yes'}}},
  'e': None},
 '{"b":[{"d":{"hi":"by"}}],"c":{"another":{"d":{"no":"yes"}}},"e":null}')

In [19]:
s = A(**aa)
s

A(b=[B(d={'hi': 'by'})], c={'another': B(d={'no': 'yes'})}, e=None)

In [20]:
s

A(b=[B(d={'hi': 'by'})], c={'another': B(d={'no': 'yes'})}, e=None)

In [21]:
print(a.__dict__)

to_del = []
for i, j in a.__dict__.items():
    if j is None:
        to_del.append(i)

    print(i, ":", j)


for i in to_del:
    a.__delattr__(i)
print(a.__dict__)

{'b': [B(d={'hi': 'by'})], 'c': {'another': B(d={'no': 'yes'})}, 'e': None}
b : [B(d={'hi': 'by'})]
c : {'another': B(d={'no': 'yes'})}
e : None
{'b': [B(d={'hi': 'by'})], 'c': {'another': B(d={'no': 'yes'})}}


In [22]:
delattr(s, "b")

In [23]:
s

A(c={'another': B(d={'no': 'yes'})}, e=None)

In [24]:
s.model_dump()

{'c': {'another': {'d': {'no': 'yes'}}}, 'e': None}